# Recommender System for Amazon Electronics

In this assignment, we will be working with the [Amazon dataset](https://drive.google.com/a/bu.edu/file/d/0B4bdB5WPHGsqT0lvTTZpbEdFS3c/view?usp=sharing). You will build a recommender system to make predictions related to reviews of Electronics products on Amazon.

Your grades will be determined by your performance on the predictive tasks as well as a brief written report about the approaches you took.

This assignment should be completed **individually**.

## Files

**train.json** 1,000,000 reviews to be used for training. It is not necessary to use all reviews for training if doing so proves too computationally intensive. The fields in this file are:

* **reviewerID** The ID of the reviewer. This is a hashed user identifier from Amazon.

* **asin** The ID of the item. This is a hashed product identifier from Amazon.

* **overall** The rating of reviewer gave the item.

* **helpful** The helpfulness votes for the review. This has 2 subfields, 'nHelpful' and 'outOf'. The latter is the total number of votes this review received. The former is the number of those that considered the review to be helpful.

* **reviewText** The text of the review.

* **summary** The summary of the review.

* **unixReviewTime** The time of the review in seconds since 1970.

**meta.json** Contains metadata of the items:

* **asin** The ID of the item.

* **categories** The category labels of the item being reviewed.

* **price** The price of the item.

* **brand** The brand of the item.

**pairs_Rating.txt** The pairs (reviewerID and asin) on which you are to predict ratings.

**pairs_Purchase.txt** The pairs on which you are to predict whether a user purchased an item or not.

**pairs_Helpful.txt** The pairs on which you are to predict helpfulness votes. A third column in this file is the total number of votes from which you should predict how many were helpful.

**helpful.json** The review data associated with the helpfulness prediction test set. The 'nHelpful' field has been removed from this data since that is the value you need to predict above. This data will only be of use for the helpfulness prediction task.

**baseline.py** A simple baseline for each task.

## Tasks

**Rating prediction** Predict people's star ratings as accurately as possible for those (reviewerID, asin) pairs in 'pairs_Rating.txt'. Accuracy will be measured in terms of the [root mean-squared error (RMSE)](http://www.kaggle.com/wiki/RootMeanSquaredError).

**Purchase prediction** Predict given a (reviewerID, asin) pair from 'pairs_Purchase.txt' whether the user purchased the item (really, whether it was one of the items they reviewed). Accuracy will be measured in terms of the [categorization accuracy](http://www.kaggle.com/wiki/HammingLoss) (1 minus the Hamming loss).

**Helpfulness prediction** Predic whether a user's review of an item will be considered helpful. The file 'pairs_Helpful.txt' contains (reviewerID, asin) pairs with a third column containing the number of votes the user's review of the item received. You must predict how many of them were helpful. Accuracy will be measured in terms of the total [absolute error](http://www.kaggle.com/wiki/AbsoluteError), i.e. you are penalized one according to the difference |nHelpful - prediction|, where 'nHelpful' is the number of helpful votes the review actually received, and 'prediction' is your prediction of this quantity.

We set up competitions on Kaggle to keep track of your results compared to those of other members of the class. The leaderboard will show your results on half of the test data, but your ultimate score will depend on your predictions across the whole dataset.
* Kaggle competition: [rating prediction](https://inclass.kaggle.com/c/cs505-amazon-rating-prediction) click here to [join](https://kaggle.com/join/rating)
* Kaggle competition: [purchase prediction](https://inclass.kaggle.com/c/cs505-amazon-purchase-prediction) click here to [join](https://kaggle.com/join/purchase)
* Kaggle competition: [helpfulness prediction](https://inclass.kaggle.com/c/cs505-amazon-helpful-prediction) click here to [join](https://kaggle.com/join/helpful)

## Grading and Evaluation

You will be graded on the following aspects.

* Your written report. This should describe the approaches you took to each of the 3 tasks. To obtain good performance, you should not need to invent new approaches (though you are more than welcome to) but rather you will be graded based on your decision to apply reasonable approaches to each of the given tasks. (**10pts** for each task)

* Your ability to obtain a solution which outperforms the baselines on the unseen portion of the test data. Obtaining full marks requires a solution which is substantially better (at least several percent) than baseline performance. (**10pts** for each task)

* Your ranking for each of the three tasks compared to other students in the class. (**10pts** for each task)

## Baselines

Simple baselines have been provided for each of the 3 tasks. These are included in 'baselines.py' among the files above. These 3 baselines operate as follows:

**Rating prediction** Returns the global average rating, or the user's average if you have seen them before in the training data.

**Purchase prediction** Finds the most popular products that account for 50% of purchases in the training data. Return '1' whenever such a product is seen at test time, '0' otherwise.

** Helpfulness prediction** Multiplies the number of votes by the global average helpfulness rate, or the user's rate if we saw this user in the training data.

Running 'baseline.py' produces 3 files containing predicted outputs. Your submission files should have the same format.

-----------------

## Dataset Citation

**Image-based recommendations on styles and substitutes** J. McAuley, C. Targett, J. Shi, A. van den Hengel *SIGIR*, 2015

**Inferring networks of substitutable and complementary products** J. McAuley, R. Pandey, J. Leskovec *Knowledge Discovery and Data Mining*, 2015

In [1]:
from collections import defaultdict
from IPython.core.display import HTML


In [2]:
def readJson(f):
    for l in open(f):
        yield eval(l)

Rating Prediction

In [3]:
allRatings = []
user_Ratings = defaultdict(list)
item_Ratings = defaultdict(list)
for l in readJson('./amazon_reviews_Electronics/train.json'):
    user,item,helpful = l['reviewerID'],l['asin'],l['helpful']['nHelpful']
    allRatings.append(l['overall'])
    user_Ratings[user] += [l['overall']]
    item_Ratings[item] += [l['overall']]

In [4]:
globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
itemAverage = {}

In [5]:
for u in user_Ratings:
    userAverage[u] = sum(user_Ratings[u]) / len(user_Ratings[u])
for i in item_Ratings:
    itemAverage[i] = sum(item_Ratings[i]) / len(item_Ratings[i])

In [6]:
count = 0
count_ui = 0
count_u = 0
count_i = 0

In [7]:
result = open("./amazon_reviews_Electronics/predictions_Rating.txt", 'w')
for l in open("./amazon_reviews_Electronics/pairs_Rating.txt"):
    if l.startswith("reviewerID"):
        result.write(l)
        continue
    u,i = l.strip().split('-')
    if u in userAverage:
        if i in itemAverage:
            count_ui += 1
            result.write(u + '-' + i + ',' + str(userAverage[u] * 0.28 + itemAverage[i] * 0.41 + globalAverage * 0.31) + '\n')
        else:
            count_u += 1
            result.write(u + '-' + i + ',' + str(userAverage[u] * 0.22 + globalAverage * 0.78) + '\n')
    elif i in itemAverage:
        count_i += 1
        result.write(u + '-' + i + ',' + str(itemAverage[i] * 0.6 + globalAverage * 0.4) + '\n')
    else:
        count += 1
        result.write(u + '-' + i + ',' + str(globalAverage) + '\n')


In [9]:
result.close()

In [10]:
print(count + count_ui + count_u + count_i)

100000


Purchase Prediction

In [11]:
itemCount = defaultdict(int)
totalPurchases = 0
priceDict = {}

In [12]:
for l in readJson('./amazon_reviews_Electronics/train.json'):
    user,item = l['reviewerID'],l['asin']
    itemCount[item] += int(l['overall'])
    totalPurchases += int(l['overall'])

In [13]:
mostPopular = [(itemCount[x], x) for x in itemCount]
mostPopular.sort()

In [15]:
result = set()
count = 0
for ic, i in mostPopular:
    count += ic
    result.add(i)
    if count > totalPurchases * 0.7: 
        break
count = 0
predictions = open("./amazon_reviews_Electronics/predictions_Purchase.txt", 'w')
for l in open("./amazon_reviews_Electronics/pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if i in result:
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")
predictions.close()

In [ ]:
Helpful Prediction

In [16]:
allHelpful = []
userHelpful = defaultdict(list)
itemRatings = defaultdict(list)
itemAverage = defaultdict(list)

In [17]:
for l in readJson('./amazon_reviews_Electronics/train.json'):
    user,item = l['reviewerID'],l['asin']
    allHelpful.append(l['helpful'])
    userHelpful[user].append(l['helpful'])
    itemRatings[item] += [l['overall']]

In [18]:
helpfulDict = {}
for l in readJson('./amazon_reviews_Electronics/helpful.json'):
    useritem, rating = l['reviewerID'] + '-' + l['asin'], float(l['overall'])
    text_len = len(l['reviewText'])
    helpfulDict[useritem] = {'rating':rating, 'text_len':text_len}

In [19]:
# calculate average item ratings   
for i in itemRatings:
    itemAverage[i] = sum(itemRatings[i]) / len(itemRatings[i])

In [20]:
averageRate = sum([x['nHelpful'] for x in allHelpful]) * 1.0 / sum([x['outOf'] for x in allHelpful])
userRate = {}

In [21]:
for user in userHelpful:
    userRate[user] = sum([x['nHelpful'] for x in userHelpful[user]]) * 1.0 / sum([x['outOf'] for x in userHelpful[user]])

In [22]:
count = 0
predictions = open("./amazon_reviews_Electronics/predictions_Helpful.txt", 'w')
for l in open("./amazon_reviews_Electronics/pairs_Helpful.txt"):
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue
    u,i,outOf = l.strip().split('-')
    useritem = u + '-' + i
    outOf = int(outOf)
    if outOf == 1:
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + '1.0' + '\n')
    elif i in itemRatings:
        tempRate = 1
        if abs(helpfulDict[useritem]['rating'] - itemAverage[i]) > 3.5:
            tempRate = 0.6
        if abs(helpfulDict[useritem]['rating'] - itemAverage[i]) > 1.75:
            count += 1
            tempRate = 0.8
        if helpfulDict[useritem]['text_len'] > 3000 or outOf > 75:
            tempRate = 0.95
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*tempRate) + '\n')
    elif u in userRate:
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*userRate[u]) + '\n')
    else:
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*averageRate) + '\n')

In [23]:
predictions.close()